In [1]:
import redis
import json

# Define connection variables
host = 'localhost'
port =  6379
password = None #'<password>'

# Connect to Redis
r = redis.Redis(host=host, port=port, password=password, decode_responses=True)
print('Connected to Redis')

r.flushdb()

Connected to Redis


True

# Dynamic Loan Spread Engine

## Description of the use case
A dynamic Loan Spread engine adjusts the loan spread based on the credit rating in real-time based on factors like User credit rating and amount of loan amount. This use case ensures that loan strategies are optimized to maximize revenue and profitability.

## Redis Data Structures Used
- **Sorted Sets**: To manage loan base_rate.
- **Hashes**: To store product pricing information and other related attributes.

In [7]:
import random

# Mock data for the use case
prd_data = {
    'product_1': {'name': 'home_loan', 'base_rate': 7.25},
    'product_2': {'name': 'personal_loan', 'base_rate': 10.5}
}

user_data = {
    'usr_1': {'name': 'Amit', 'credit_rating': 825},
    'usr_2': {'name': 'Sunil', 'credit_rating': 675},
    'usr_3': {'name': 'Dilip', 'credit_rating': 840},
    'usr_4': {'name': 'Akash', 'credit_rating': 795},
    'usr_5': {'name': 'Vikas', 'credit_rating': 775}
}

# Use Hashes to store product rate information
for product_id, product_info in prd_data.items():
    r.hset(product_id, mapping=product_info)
    print(f"Product Info : {product_id} : {product_info}")

# Use Hashes to store user information
for user_id, user_info in user_data.items():
    r.hset(user_id, mapping=user_info)
    print(f"User Info : {user_id} : {user_info}")

# Dynamic pricing algorithm
def dynamic_spread_engine(credit_rating, base_rate):
    new_price = base_rate + ((900-credit_rating) * 0.01)  
    return new_price

# iterate over users and calculate loan rates
for user_id in user_data.keys():
    credit_rating = int(r.hget(user_id, 'credit_rating'))
    # Update prices for all products
    for product_id in prd_data.keys():
        base_rate = float(r.hget(product_id, 'base_rate'))
        product_name = r.hget(product_id, 'name')
        updated_rate = dynamic_spread_engine(credit_rating, base_rate)
        r.hset(user_id, product_name, updated_rate)
        #print(f"Updated rate for {product_id}: {updated_rate}")

# Print the updated pricing information
for user_id in user_data.keys():
    print(f"New Updated Loan Spread :{user_id}: {r.hgetall(user_id)}")




Product Info : product_1 : {'name': 'home_loan', 'base_rate': 7.25}
Product Info : product_2 : {'name': 'personal_loan', 'base_rate': 10.5}
User Info : usr_1 : {'name': 'Amit', 'credit_rating': 825}
User Info : usr_2 : {'name': 'Sunil', 'credit_rating': 675}
User Info : usr_3 : {'name': 'Dilip', 'credit_rating': 840}
User Info : usr_4 : {'name': 'Akash', 'credit_rating': 795}
User Info : usr_5 : {'name': 'Vikas', 'credit_rating': 775}
usr_1
825
usr_2
675
usr_3
840
usr_4
795
usr_5
775
New Updated Loan Spread :usr_1: {'name': 'Amit', 'credit_rating': '825', 'home_loan': '8.0', 'personal_loan': '11.25'}
New Updated Loan Spread :usr_2: {'name': 'Sunil', 'credit_rating': '675', 'home_loan': '9.5', 'personal_loan': '12.75'}
New Updated Loan Spread :usr_3: {'name': 'Dilip', 'credit_rating': '840', 'home_loan': '7.85', 'personal_loan': '11.1'}
New Updated Loan Spread :usr_4: {'name': 'Akash', 'credit_rating': '795', 'home_loan': '8.3', 'personal_loan': '11.55'}
New Updated Loan Spread :usr_5: 